In [ ]:
import requests, json, logging, time, datetime, math, base64
from typing import Literal, List, Union
from notebookutils import mssparkutils


class fabric_rest():
    def __init__(self, audience:str='pbi'):
        self.header = self.create_header(audience)

    def create_header(self, audience:str='pbi') -> dict:
        token = mssparkutils.credentials.getToken(audience)
        return {'Authorization': f'Bearer {token}', 'Content-type': 'application/json'}


    def _base64_decode_bytes(self, base64String:str) -> bytes:
        return base64.b64decode(base64String.encode('utf-8'))


    def base64_decode(self, base64String:str) -> str:
        return self._base64_decode_bytes(base64String).decode('utf-8')
    

    def _base64_encode_bytes(self, string:str) -> bytes:
        return base64.b64encode(string.encode('utf-8'))


    def _base64_encode(self, string:str) -> str:
        return self._base64_encode_bytes(string).decode('utf-8')


    def request(self, method:str, url:str, body:dict=None) -> List[requests.Response]:
        logger.info(f'request - {method} - {url} - {body}')
        responseList = []
        def make_request(method:str, url:str, body:dict=None):
            try:
                response = requests.request(method=method, url=url, headers=self.header, data=json.dumps(body))
                logger.debug(response.json())
                response.raise_for_status()
                logger.debug(f"Response - {response.status_code}")
                if response.status_code == 202:
                    response = self._response_long_running(response=response)
                
                responseList.append(response)
                if response.json().get('continuationUri') is not None and response.json().get('continuationToken') is not None:
                    logger.info(f'continuationUri - {response.json().get("continuationUri")}')
                    make_request(method='get', url=f'{response.json().get("continuationUri")}')
            except requests.exceptions.HTTPError as errh:
                if 'Request is blocked by the upstream service until:' in errh.response.json()['message']:
                    blockedDatetime = datetime.datetime.strptime(errh.response.json()['message'].split('Request is blocked by the upstream service until: ')[1], '%m/%d/%Y %I:%M:%S %p')
                    sleepDuration = math.ceil((blockedDatetime - datetime.datetime.now(datetime.UTC).replace(tzinfo=None)).total_seconds())
                    logger.info(f"sleeping for {sleepDuration} seconds")
                    time.sleep(sleepDuration) # pause until we can make the request again
                    make_request(method=method, url=url, body=body)
                else:
                    raise Exception("Http Error:", errh.response.text)
            except requests.exceptions.ConnectionError as errc:
                raise Exception("Error Connecting:", errc.response.headers)
            except requests.exceptions.Timeout as errt:
                raise Exception("Timeout Error:", errt.response.text)
            except requests.exceptions.RequestException as err:
                raise Exception(response.status_code, err)
                
        make_request(method=method, url=url, body=body)
        logger.info(f'responseList - {responseList}')
        return responseList


    def _response_long_running(self, response:requests.Response) -> requests.Response:
        responseLocation = response.headers.get('Location')
        # Will pause 5 unique times before failing
        for _ in range(5):
            responseStatus = self.request(method='get', url=responseLocation)[0] # Just get the first item in the list because it should only have one item.
            if responseStatus.json().get('status') != 'Succeeded':
                logger.info(f'Operation {response.headers.get("x-ms-operation-id")} is not ready. Waiting for {response.headers.get("Retry-After")} seconds.')
                time.sleep(int(response.headers.get('Retry-After')))
            else:
                if responseStatus.headers.get('Location') is None:
                    logger.info('Long running operation has completed. No result to be returned')
                    return responseStatus
                else:
                    logger.info('Payload is ready. Requesting the result.')
                    responseResult = self.request(method='get', url=responseStatus.headers.get('Location'))[0] # Just get the first item in the list because it should only have one item.
                    return responseResult
                

    def response_parse(self, response:List[requests.Response]) -> dict:
        responseParsed = [response.json() for response in response]
        return responseParsed
    

    def response_list_unravel(self, responseList:List[requests.Response], param:str='value') -> list:
        responseUnraveled = []
        for responseItem in self.response_parse(responseList):
            if responseItem.get(param) is not None:
                for response in responseItem.get(param):
                    responseUnraveled.append(response)
            else:
                responseUnraveled.append(responseItem)
        return responseUnraveled
    

    def response_build_parameters(self, **paramaters:dict) -> str:
        parameterString = '&'.join([f'{k}={v}' for k,v in paramaters.items() if v is not None])
        responseParameterString = f'?{parameterString}' if parameterString != '' else ''
        return responseParameterString


    def capacity_list_response(self) -> requests.Response:
        response = self.request(method='get', url=f'https://api.fabric.microsoft.com/v1/capacities')
        return response
    

    def capacity_list(self) -> list:
        capacityList = self.response_list_unravel(responseList=self.capacity_list_response(), param='value')
        return capacityList
    

    def capacity_get(self, capacityName:str) -> dict:
        capacity = [capacity for capacity in self.capacity_list() if capacity.get('displayName') == capacityName][0]
        return capacity
    

    def workspace_list_response(self, capacity:str=None, name:str=None, state:str=None, type:str=None) -> List[requests.Response]:
        logger.info('workspace_list_response')
        url = f'https://api.fabric.microsoft.com/v1/workspaces{self.response_build_parameters(capacity=capacity, name=name, state=state, type=type)}'
        response = self.request(method='get', url=url)
        return response
    

    def workspace_list(self, capacity:str=None, name:str=None, state:str=None, type:str=None) -> list:
        logger.info('workspace_list')
        workspaceList = self.response_list_unravel(responseList=self.workspace_list_response(capacity=capacity, name=name, state=state, type=type), param='value')
        return workspaceList


    def workspace_create_response(self, workspaceName:str, capacityName:str, description:str=None) -> requests.Response:
        capacityId = self.capacity_get(capacityName=capacityName).get('id')
        body = {
            "displayName": workspaceName
            ,"capacityId": capacityId
            ,**({"description": description} if description is not None else {})
        }
        response = self.request(method='post', url='https://api.fabric.microsoft.com/v1/workspaces', body=body)
        return response


    def workspace_create(self, workspaceName:str, capacityName:str, description:str=None) -> dict:
        response = self.response_list_unravel(self.workspace_create_response(workspaceName=workspaceName, capacityName=capacityName, description=description), param='value')
        return response
    

    def workspace_delete(self, workspaceName:str) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        response = self.request(method='delete', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}')
        return response
    

    def workspace_assign_capacity(self, workspaceName:str, capacityName:str) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        capacityId = self.capacity_get(capacityName=capacityName).get('id')
        body = {"capacityId": capacityId}
        response = self.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/assignToCapacity', body=body)
        return response
    

    def workspace_add_role_assignment(self, workspaceName:str, principalName:str, role:Literal['Admin', 'Contributor', 'Member', 'Viewer']) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        principal = {'id': self.principal_get_id(principalName=principalName), "type": "User"}
        body = {'principal': principal, 'role': role}
        response = self.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/roleAssignments', body=body)
        return response
    
    
    def workspace_delete_role_assignment(self, workspaceName:str, principalName:str) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        principalId = {'id': self.principal_get_id(principalName=principalName), "type": "User"}
        response = self.request(method='delete', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/roleAssignments/{principalId}')
        return response
    

    def workspace_get_response(self, workspaceName:str=None, workspaceId:str=None) -> str:
        logger.info(f'workspace_get_response: {workspaceName=} | {workspaceId=}')
        if workspaceId is None and workspaceName is not None:
            workspaceId = [workspace.get('id') for workspace in self.workspace_list() if workspace.get('displayName') == workspaceName][0]
        elif workspaceName is None and workspaceId is None:
            raise Exception('Either workspaceName or workspaceId must be provided')
        logger.info(f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}')
        response = self.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}')
        return response
    

    def workspace_get(self, workspaceName:str=None, workspaceId:str=None) -> str:
        logger.info(f'workspace_get: {workspaceName=} | {workspaceId=}')
        response = self.workspace_get_response(workspaceName=workspaceName, workspaceId=workspaceId)[0].json()
        return response
    

    def workspace_get_id(self, workspaceName:str) -> str:
        logger.info(f'workspace_get_id: {workspaceName=}')
        workspaceId = [workspace.get('id') for workspace in self.workspace_list() if workspace.get('displayName') == workspaceName][0]
        return workspaceId
    

    def _workspace_update_metadata(self, workspaceName:str, workspaceNameTarget:str=None, workspaceDescription:str=None) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        body = {k:v for k,v in {'displayName':workspaceNameTarget, 'description': workspaceDescription}.items() if v != ''}
        response = self.request(method='patch', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}', body=body)
        return response


    def workspace_rename(self, workspaceName:str, workspaceNameTarget:str) -> requests.Response:
        response = self._workspace_update_metadata(workspaceName=workspaceName, workspaceNameTarget=workspaceNameTarget)
        return response
    

    def workspace_set_description(self, workspaceName:str, workspaceDescription:str) -> requests.Response:
        response = self._workspace_update_metadata(workspaceName=workspaceName, workspaceDescription=workspaceDescription)
        return response
    

    def pipeline_list_response(self, workspaceName:str) -> str:
        pipelineResponse = self.item_list_response(workspaceName=workspaceName, itemType='DataPipeline')
        return pipelineResponse
    

    def pipeline_list(self, workspaceName:str) -> list:
        pipelineList = self.response_list_unravel(self.pipeline_list_response(workspaceName=workspaceName), param='value')
        return pipelineList
    

    def pipeline_create(self, workspaceName:str, pipelineName:str, pipelineDefinition:dict) -> requests.Response:
        response = self.item_create(workspaceName=workspaceName, itemName=pipelineName, itemType='DataPipeline', itemDefinition=pipelineDefinition)
        return response
    
    
    def pipeline_delete(self, workspaceName:str, pipelineName:str) -> requests.Response:
        response = self.item_delete(workspaceName=workspaceName, itemName=pipelineName)
        return response
    

    def pipeline_update_metadata(self, workspaceName:str, pipelineName:str, pipelineNameNew:str=None, pipelineDescription:str=None) -> str:
        body = {k:v for k,v in {'displayName':pipelineNameNew, 'description': pipelineDescription}.items() if v != ''}
        response = self.item_update_metadata(workspaceName=workspaceName, itemName=pipelineName, body=body)
        return response
    

    def pipeline_update_definition(self, workspaceName:str, pipelineName:str, pipelineDefinition:dict) -> str:
        response = self.item_update_definition(workspaceName=workspaceName, itemName=pipelineName, definition=pipelineDefinition)
        return response


    def pipeline_run(self, workspaceName:str, pipelineName:str) -> requests.Response:
        response = self.item_run_job(workspaceName=workspaceName, itemName=pipelineName, jobType='Pipeline')
        return response
    

    def pipeline_get_run_instance(self, workspaceName:str, pipelineName:str, jobInstanceId:str) -> requests.Response:
        response = self.item_get_job_instance(workspaceName=workspaceName, itemName=pipelineName, jobInstanceId=jobInstanceId)
        return response


    def pipeline_cancel_run_instance(self, workspaceName:str, pipelineName:str, jobInstanceId:str) -> requests.Response:
        response = self.item_cancel_job_instance(workspaceName=workspaceName, itemName=pipelineName, jobInstanceId=jobInstanceId)
        return response
    

    def pipeline_get_object(self, workspaceName:str, pipelineName:str) -> str:
        lakehouseId = self.item_get_object(workspaceName=workspaceName, itemName=pipelineName, itemType='DataPipeline')
        return lakehouseId
    

    # https://learn.microsoft.com/en-us/rest/api/fabric/admin/items/list-items?tabs=HTTP
    def pipeline_get_id(self, workspaceName:str, pipelineName:str) -> str:
        pipelineId = self.item_get_id(workspaceName=workspaceName, itemName=pipelineName, itemType='DataPipeline')
        return pipelineId


    # https://learn.microsoft.com/en-us/rest/api/fabric/core/items/get-item-definition?tabs=HTTP
    def pipeline_get_definition(self, workspaceName:str, pipelineName:str) -> list:
        itemDefinition = self.item_get_definition(workspaceName=workspaceName, itemName=pipelineName, itemType='DataPipeline')
        return itemDefinition
    

    def pipeline_get_definition_parts(self, workspaceName:str, pipelineName:str) -> list:
        itemDefinitionParts = self.item_get_definition_parts(workspaceName=workspaceName, itemName=pipelineName, itemType='DataPipeline')
        return itemDefinitionParts


    # https://learn.microsoft.com/en-us/rest/api/fabric/core/items/create-item?tabs=HTTP
    def pipeline_clone(self, workspaceNameSource:str, pipelineNameSource:str, workspaceNameTarget:str, pipelineNameTarget:str) -> requests.Response:
        pipelineDefinition = self.pipeline_get_definition(workspaceName=workspaceNameSource, pipelineName=pipelineNameSource)
        response = self.item_create(workspaceName=workspaceNameTarget, itemName=pipelineNameTarget, itemType='DataPipeline', itemDefinition=pipelineDefinition)
        return response


    # https://learn.microsoft.com/en-us/rest/api/fabric/admin/items/list-items?tabs=HTTP
    def notebook_get_id(self, workspaceName:str, notebookName:str) -> str:
        notebookId = self.item_get_id(workspaceName=workspaceName, itemName=notebookName, itemType='Notebook')
        return notebookId


    def notebook_get_item_definition(self, workspaceName:str, notebookName:str) -> requests.Response:
        definition = self.item_get_definition(workspaceName=workspaceName, itemName=notebookName, itemType='Notebook', format='ipynb')
        return definition


    def notebook_get_item_definition_decoded(self, workspaceName:str, notebookName:str) -> requests.Response:
        itemDefinition = self.notebook_get_item_definition(workspaceName=workspaceName, notebookName=notebookName)
        definition = self.base64_decode(itemDefinition.get('definition').get('parts')[0].get('payload'))
        return definition


    def notebook_clone(self, workspaceNameSource:str, notebookNameSource:str, workspaceNameTarget:str, notebookNameTarget:str) -> requests.Response:
        notebook_get_definition = self.notebook_get_item_definition(workspaceName=workspaceNameSource, notebookName=notebookNameSource)
        response = self.item_create(workspaceName=workspaceNameTarget, itemName=notebookNameTarget, itemType='Notebook', itemDefinition=notebook_get_definition)
        return response
    

    def notebook_delete(self, workspaceName:str, notebookName:str) -> str:
        response = self.item_delete(workspaceName=workspaceName, itemName=notebookName)
        return response
    
    def item_get_response(self, workspaceName:str, itemName:str, itemType:Literal['Dashboard', 'DataPipeline', 'Datamart', 'Eventstream', 'KQLDataConnection', 'KQLDatabase', 'KQLQueryset', 'Lakehouse', 'MLExperiment', 'MLModel', 'MirroredWarehouse', 'Notebook', 'PaginatedReport', 'Report', 'SQLEndpoint', 'SemanticModel', 'SparkJobDefinition', 'Warehouse']) -> List[requests.Response]:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        itemId = self.item_get_id(workspaceName=workspaceName, itemName=itemName, itemType=itemType)
        response = self.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}')
        return response
    
    def item_get_by_id_response(self, workspaceName:str, itemId:str) -> List[requests.Response]:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        response = self.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}')
        return response
    
    def item_get_by_id(self, workspaceName:str, itemId:str) -> List[requests.Response]:
        item = self.item_get_by_id_response(workspaceName=workspaceName, itemId=itemId)[0].json()
        return item
    

    def item_get(self, workspaceName:str, itemName:str, itemType:Literal['Dashboard', 'DataPipeline', 'Datamart', 'Eventstream', 'KQLDataConnection', 'KQLDatabase', 'KQLQueryset', 'Lakehouse', 'MLExperiment', 'MLModel', 'MirroredWarehouse', 'Notebook', 'PaginatedReport', 'Report', 'SQLEndpoint', 'SemanticModel', 'SparkJobDefinition', 'Warehouse']) -> List[requests.Response]:
        item_get_response = self.item_get_response(workspaceName=workspaceName, itemName=itemName, itemType=itemType)[0].json()
        return item_get_response
    

    def item_list_response(self, workspaceName:str, itemType:Literal['Dashboard', 'DataPipeline', 'Datamart', 'Eventstream', 'KQLDataConnection', 'KQLDatabase', 'KQLQueryset', 'Lakehouse', 'MLExperiment', 'MLModel', 'MirroredWarehouse', 'Notebook', 'PaginatedReport', 'Report', 'SQLEndpoint', 'SemanticModel', 'SparkJobDefinition', 'Warehouse']=None) -> List[requests.Response]:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        filter = f'?type={itemType}' if itemType else ''
        response = self.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items{filter}')
        return response


    def item_list(self, workspaceName:str, itemType:Literal['Dashboard', 'DataPipeline', 'Datamart', 'Eventstream', 'KQLDataConnection', 'KQLDatabase', 'KQLQueryset', 'Lakehouse', 'MLExperiment', 'MLModel', 'MirroredWarehouse', 'Notebook', 'PaginatedReport', 'Report', 'SQLEndpoint', 'SemanticModel', 'SparkJobDefinition', 'Warehouse']=None) -> list:
        item_list_response = self.item_list_response(workspaceName=workspaceName, itemType=itemType)
        item_list = self.response_list_unravel(item_list_response, param='value')
        return item_list
    

    def item_list_admin_response(self, workspaceName:str=None, itemType:Literal['Dashboard', 'DataPipeline', 'Datamart', 'Eventstream', 'KQLDataConnection', 'KQLDatabase', 'KQLQueryset', 'Lakehouse', 'MLExperiment', 'MLModel', 'MirroredWarehouse', 'Notebook', 'PaginatedReport', 'Report', 'SQLEndpoint', 'SemanticModel', 'SparkJobDefinition', 'Warehouse']=None
                        ,capacity:str=None, state:str=None, type:str=None, workspaceId:str=None) -> list:
        logger.info(f'item_list_admin_response: {workspaceName=}, {itemType=}')
        response = self.request(method='get', url=f'https://api.fabric.microsoft.com/v1/admin/items{self.response_build_parameters(capacity=capacity, state=state, type=type, workspaceId=workspaceId)}')
        return response
    

    def item_list_admin(self, workspaceName:str=None, itemType:Literal['Dashboard', 'DataPipeline', 'Datamart', 'Eventstream', 'KQLDataConnection', 'KQLDatabase', 'KQLQueryset', 'Lakehouse', 'MLExperiment', 'MLModel', 'MirroredWarehouse', 'Notebook', 'PaginatedReport', 'Report', 'SQLEndpoint', 'SemanticModel', 'SparkJobDefinition', 'Warehouse']=None
                        ,capacity:str=None, state:str=None, type:str=None, workspaceId:str=None) -> list:
        logger.info(f'item_list_admin: {workspaceName=}, {itemType=}')
        responseParsed = self.response_list_unravel(self.item_list_admin_response(workspaceName=workspaceName, itemType=itemType, capacity=capacity, state=state, type=type, workspaceId=workspaceId), param='itemEntities')
        return responseParsed

    def item_get_object(self, workspaceName:str, itemName:str, itemType:str=None) -> dict:
        item_list = self.item_list(workspaceName=workspaceName, itemType=itemType)
        try:
            artifact = [item for item in item_list if item.get('displayName') == itemName][0]
            return artifact
        except IndexError as ie:
            raise(f'Item {itemName} not found in workspace {workspaceName} - {ie}')


    def item_get_id(self, workspaceName:str, itemName:str, itemType:str=None) -> str:
        artifactObject = self.item_get_object(workspaceName=workspaceName, itemName=itemName, itemType=itemType)
        artifactId = artifactObject.get('id') if artifactObject else None
        return artifactId


    def item_get_definition_response(self, workspaceName:str, itemName:str, itemType:str=None, format=None) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        itemId = self.item_get_id(workspaceName=workspaceName, itemName=itemName, itemType=itemType)
        filter = f'?format={format}' if format else ''
        response = self.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}/getDefinition{filter}')
        return response
    

    def item_get_definition(self, workspaceName:str, itemName:str, itemType:str='', format=None) -> dict:
        response = self.response_list_unravel(self.item_get_definition_response(workspaceName=workspaceName, itemName=itemName, itemType=itemType), param=None)[0]
        return response
    

    def item_get_definition_parts(self, workspaceName:str, itemName:str, itemType:str='', format=None) -> list:
        response = self.item_get_definition(workspaceName=workspaceName, itemName=itemName, itemType=itemType, format=format).get('definition').get('parts')
        return response


    def item_create(self, workspaceName:str, itemName:str, itemType:str, itemDefinition:dict=None) -> requests.Response:
        logger.info(f'item_create - {workspaceName=}:{itemName=}:{itemType=}')
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        body = {"displayName": itemName
                ,"type": itemType
                ,**(itemDefinition if itemDefinition is not None else {})
                # ,**({'definition': itemDefinition.get('definition')} if itemDefinition is not None else {}) ## This can be None when creating a Lakehouse
                }
        response = self.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items', body=body)
        return response
    

    # This one does not work yet!! (for lakehouse)
    def item_delete(self, workspaceName:str, itemName:str):
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        itemId = self.item_get_id(workspaceName=workspaceName, itemName=itemName)
        logger.info(f'item_delete - {workspaceName=}:{workspaceId=} - {itemName=}:{itemId}')
        response = self.request(method='delete', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}')
        return response


    def item_update_metadata(self, workspaceName:str, itemName:str, body:dict) -> str:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        itemId = self.item_get_id(workspaceName=workspaceName, itemName=itemName)
        response = self.request(method='patch', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}', body=body)
        return response
    

    def item_run_job(self, workspaceName:str, itemName:str, jobType:Literal['Pipeline']) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        itemId = self.item_get_id(workspaceName=workspaceName, itemName=itemName)
        logger.info('item_run_job')
        filter = {f'?jobType={jobType}' if jobType else ''}
        response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}/jobs/instances{filter}', headers=self.header)
        return response.headers.get('Location').split('jobs/instances/')[-1]
    

    def item_get_job_instance_response(self, workspaceName:str, itemName:str, jobInstanceId:str) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        itemId = self.item_get_id(workspaceName=workspaceName, itemName=itemName)
        response = self.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}/jobs/instances/{jobInstanceId}')
        return response
    
    
    def item_get_job_instance(self, workspaceName:str, itemName:str, jobInstanceId:str) -> dict:
        response = self.item_get_job_instance_response(workspaceName=workspaceName, itemName=itemName, jobInstanceId=jobInstanceId).json()
        return response
    
    
    def item_cancel_job_instance(self, workspaceName:str, itemName:str, jobInstanceId:str) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        itemId = self.item_get_id(workspaceName=workspaceName, itemName=itemName)
        response = self.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}/jobs/instances/{jobInstanceId}/cancel')
        return response


    def lakehouse_delete_shortcut_response(self, workspaceName:str, lakehouseName:str, shortcutPath:str, shortcutName:str) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        itemId = self.lakehouse_get_id(workspaceName=workspaceName, lakehouseName=lakehouseName)
        # https://learn.microsoft.com/en-us/rest/api/fabric/core/onelake-shortcuts/delete-shortcut?tabs=HTTP
        response = self.request(method='delete', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}/shortcuts/{shortcutPath}/{shortcutName}')
        return response
    
    def lakehouse_delete_shortcut(self, workspaceName:str, lakehouseName:str, shortcutPath:str, shortcutName:str) -> requests.Response:
        response = self.lakehouse_delete_shortcut_response(workspaceName=workspaceName, lakehouseName=lakehouseName, shortcutPath=shortcutPath, shortcutName=shortcutName)[0].json()
        return response


    def lakehouse_get_shortcut_response(self, workspaceName:str, lakehouseName:str, shortcutPath:str, shortcutName:str) -> requests.Response:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        itemId = self.lakehouse_get_id(workspaceName=workspaceName, lakehouseName=lakehouseName)
        # https://learn.microsoft.com/en-us/rest/api/fabric/core/onelake-shortcuts/get-shortcut?tabs=HTTP
        response = self.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}/shortcuts/{shortcutPath}/{shortcutName}')
        return response
    

    def lakehouse_get_shortcut(self, workspaceName:str, lakehouseName:str, shortcutPath:str, shortcutName:str) -> requests.Response:
        response = self.lakehouse_get_shortcut_response(workspaceName=workspaceName, lakehouseName=lakehouseName, shortcutPath=shortcutPath, shortcutName=shortcutName)[0].json()
        return response
    

    def _lakehouse_create_shortcut(self, workspaceId:str, itemId:str, body:dict) -> requests.Response:
        response = self.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{itemId}/shortcuts?shortcutConflictPolicy=GenerateUniqueName', body=body)
        return response


    def lakehouse_list_response(self, workspaceName:str) -> requests.Response:
        lakehouseResponse = self.item_list_response(workspaceName=workspaceName, itemType='Lakehouse')
        return lakehouseResponse
    

    def lakehouse_list(self, workspaceName:str) -> requests.Response:
        logger.info(f'lakehouse_list: {workspaceName=}')
        lakehouseResponse = self.response_list_unravel(self.lakehouse_list_response(workspaceName=workspaceName), param='value')
        return lakehouseResponse
    

    def lakehouse_get_object(self, workspaceName:str, lakehouseName:str) -> str:
        lakehouseId = self.item_get_object(workspaceName=workspaceName, itemName=lakehouseName, itemType='Lakehouse')
        return lakehouseId
    

    def lakehouse_get_id(self, workspaceName:str, lakehouseName:str) -> str:
        lakehouseId = self.item_get_id(workspaceName=workspaceName, itemName=lakehouseName, itemType='Lakehouse')
        return lakehouseId
    


In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

fr = fabric_rest()

print(fr.notebook_get_item_definition_decoded(workspaceName='WS_Python_SDK', notebookName='Notebook 1'))